In [1]:
import pygame
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
import random

from keras.layers import Dense,Input,Flatten
from keras.models import Model
from keras.models import model_from_json

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


Using TensorFlow backend.


In [2]:
def modelRoot(inputX,classes):
    X=Flatten()(inputX)
    X=Dense(16,activation='relu')(X)
    X=Dense(8,activation='relu')(X)
    X=Dense(classes,activation=None)(X)
    return X

In [3]:
inputShape= Input((5,1))
mRoot=modelRoot(inputShape,2)
model=Model(inputs=inputShape,outputs=mRoot)
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5, 1)              0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                96        
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 18        
Total params: 250
Trainable params: 250
Non-trainable params: 0
_________________________________________________________________


In [4]:
json_file = open('model2a.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("modelWeight2a.h5")

In [5]:
def nextAction(dataQ,explorationRate,willLearn):
    alives=dataQ.shape[0]
    ans=np.zeros(alives)
    bb=0
    for it in range(totalBalls):
        if ballAlive[it]==0:
            continue
        if random.random() > explorationRate or not willLearn:
            ans[bb]=np.argmax(dataQ[bb])
        else:
            rn=random.random()
            if rn>biasRn[it]:
                ans[bb]=0
            else :
                ans[bb]=1
        bb+=1
    return ans

In [6]:
iterations=1
winHeight=700
winWidth=500
out=False

willLearn=False
explorationRate=1.0
explorationDelta=1.0/iterations
lamda=0.95

for gen in range(iterations):
    
    totalObjects=5
    totalBalls=totalObjects
    ballX=np.zeros(totalBalls)
    ballY=np.zeros(totalBalls)
    vX=np.zeros(totalBalls)
    vY=np.zeros(totalBalls)
    unitX=np.zeros(totalBalls)
    unitY=np.zeros(totalBalls)
    scores=np.zeros(totalBalls)
    biasRn=np.zeros(totalBalls)
    justNow=np.zeros(totalBalls)
    score=np.zeros(totalBalls)
    ballAlive=np.ones(totalBalls)
    rewards=np.ones(totalBalls)
    ups=np.ones(totalBalls)
    downs=np.ones(totalBalls)
    lefts=np.ones(totalBalls)
    rights=np.ones(totalBalls)
    unit=4
    ballLen=20
    
    totalBars=totalObjects
    posX=np.zeros(totalBars)
    posY=np.zeros(totalBars)
    barWidth=80
    barHeight=20
    
    for it in range(totalBalls):
        ballX[it]=random.random()*(winWidth-ballLen-20)+10
        ballY[it]=random.random()*(winHeight)*0.1+10
        vX[it]=random.random()*0.8
        vY[it]=1-vX[it]
        unitX[it]=1
        unitY[it]=1
        if random.random()>0.5:
            unitX[it]=-unitX[it]
        biasRn[it]=0.5#random.random()
            
    for it in range(totalBars):
        posY[it]=600
        posX[it]=random.random()*(winWidth-barWidth-5)+5
            
    pygame.init()
    pygame.font.init()
    win=pygame.display.set_mode((winWidth,winHeight))
    pygame.display.set_caption('Tennis')
    CC=pygame.time.Clock()
    myfont=pygame.font.SysFont('Comic Sans MS',15)

    done = False
    while not done:
        CC.tick(50)
        win.fill((255,255,255))
        for event in pygame.event.get():
            if event.type==pygame.QUIT:
                done=True
                out=True
        keys=pygame.key.get_pressed()
        if keys[pygame.K_RIGHT]:
            for it in range(totalBars):
                if ballAlive[it]==1:
                    if posX[it]+unit+barWidth < winWidth:
                        posX[it]+=unit
        if keys[pygame.K_LEFT]:
            for it in range(totalBars):
                if ballAlive[it]:
                    if posX[it]+unit>0:
                        posX[it]-=unit
                        
        ########################## reinforcement learning old state info
        alives=0
        oldStates=[]
        for it in range(totalBalls):
            if ballAlive[it]==0:
                continue
            alives+=1
            rewards[it]=0
            
            #############
       
            oldState=[]
            oldState.append(posX[it]/winWidth)
            oldState.append(ballX[it]/winWidth)
            oldState.append((600-ballY[it])/winHeight)
            oldState.append(vX[it]*unitX[it])
            oldState.append(vY[it]*unitY[it])
            oldState=np.asarray(oldState)
            oldStates.append(oldState)
        oldStates=np.asarray(oldStates)
        oldStates=oldStates.reshape(oldStates.shape[0],oldStates.shape[1],1)
        oldQs=model.predict(oldStates)
        ############  next Action
        action=nextAction(oldQs,explorationRate,willLearn)
        ##########################
        
        ##applying actions
        cur=0
        for it in range(totalBalls):
            if ballAlive[it]==1:
                if action[cur]==0:
                    if posX[it]+unit>0:
                        posX[it]-=unit
                else :
                    if posX[it]+unit+barWidth < winWidth:
                        posX[it]+=unit
                cur+=1
        ###########################
        
        ##updating postions
        for it in range(totalBalls):
            if ballAlive[it]==1:
                nX=vX[it]*unitX[it]*unit
                nY=vY[it]*unitY[it]*unit
                '''if ballX[it]<250 and ballX[it]+nX>=250:
                    lefts[it]=1
                if ballX[it]>250 and ballX[it]+nX<=250:
                    rights[it]=1
                if ballY[it]<250 and ballY[it]+nY>=250:
                    ups[it]=1
                if ballY[it]>250 and ballY[it]+nY<=250:
                    downs[it]=1'''
                ballX[it]+=nX
                ballY[it]+=nY
        
        ##intersections
        for it in range(totalBalls):
            if ballAlive[it]==1:
                if ballX[it]<0:
                    unitX[it]=-unitX[it]
                    ballX[it]=0
                if ballX[it]+ballLen>winWidth:
                    unitX[it]=-unitX[it]
                    ballX[it]=winWidth-ballLen
                if ballY[it]<0:
                    unitY[it]=-unitY[it]
                    ballY[it]=0
                if ballY[it]+ballLen>winHeight:
                    unitY[it]=-unitY[it]
                if ballY[it]+ballLen>posY[it] and(
                    ballX[it]+ballLen<posX[it] or
                     ballX[it]> posX[it]+barWidth):
                    ballAlive[it]=0
                    rewards[it]=-1
                    justNow[it]=1
                    continue
                if ballY[it]+ballLen>=posY[it] and(
                    ballX[it]+ballLen>=posX[it] and
                     ballX[it]<= posX[it]+barWidth):
                    unitY[it]=-unitY[it]
                    mid=posX[it]+(barWidth/2)
                    midBall=ballX[it]+(ballLen/2)
                    dd=midBall-mid
                    unitX[it]=1
                    ballY[it]=600-ballLen
                    if dd<0:
                        unitX[it]=-1
                    dd=abs(dd)
                    far=(barWidth+ballLen+20)/2
                    vX[it]=dd/far
                    vY[it]=1-vX[it]
                    rewards[it]=1
                    score[it]+=1
                    ##### version 2
                    #ballAlive[it]=0
                    #justNow[it]=1
                
        ########################## reinforcement learning new state info
        newStates=[]
        for it in range(totalBalls):
            if ballAlive[it]==0 and justNow[it]==0:
                continue
                    
            #############
            newState=[]
            newState.append(posX[it]/winWidth)
            newState.append(ballX[it]/winWidth)
            newState.append((600-ballY[it])/winHeight)
            newState.append(vX[it]*unitX[it])
            newState.append(vY[it]*unitY[it])
            newStates.append(newState)
        newStates=np.asarray(newStates)
        newStates=newStates.reshape(newStates.shape[0],newStates.shape[1],1)
        newQs=model.predict(newStates)
        ##########################

        ########################## reinforcement learning train
        cur=0
        
        for it in range(totalBalls):
            if ballAlive[it]==0 and justNow[it]==0:
                continue
            okk=rewards[it] + lamda * np.amax(newQs[cur])
            oldQs[cur,int(action[cur])] = okk
            cur+=1
        if willLearn:
            model.fit(oldStates,oldQs,verbose=0)
        #############################################
        
        
        ##dwraing balls
        mm=0
        for it in range(totalBalls):
            justNow[it]=0
            if ballAlive[it]==1 and mm<2000:
                mm+=1
                pygame.draw.rect(win,(0,0,0),(ballX[it],ballY[it],ballLen,ballLen))
        mm=0
        for it in range(totalBars):
            if ballAlive[it]==1 and mm<2000:
                mm+=1
                pygame.draw.rect(win,(0,0,0),(posX[it],posY[it],barWidth,barHeight))
        
        ##showing data
        highest=0
        cntAlive=0
        hmm=0.0
        for it in range(totalBalls):
            if ballAlive[it]==1:
                cntAlive+=1
            highest=max(highest,score[it])
            #hmm+=score[it]
        
        text=myfont.render('Score: '+str(int(highest)),1,(0,0,0))
        win.blit(text,(winWidth-80,650))
        
        text=myfont.render('Alives: '+str(int(cntAlive)),1,(0,0,0))
        win.blit(text,(200,650))
        
        text=myfont.render('Gen: '+str(int(gen)),1,(0,0,0))
        win.blit(text,(20,650))
        
        dead=0
        for it in range(totalBalls):
            if ballAlive[it]==0:
                dead+=1
        if dead==totalObjects:
            done=True
        pygame.display.update()
        
    if explorationRate>0:
        explorationRate-=explorationDelta
    print('avg.-> '+str(highest))
    if out:
        break
pygame.quit()

avg.-> 4.0


In [7]:
'''# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("modelWeight.h5")
print("Saved model to disk")'''

'# serialize model to JSON\nmodel_json = model.to_json()\nwith open("model.json", "w") as json_file:\n    json_file.write(model_json)\n# serialize weights to HDF5\nmodel.save_weights("modelWeight.h5")\nprint("Saved model to disk")'